In [1]:
import requests
import json
import os
import pandas as pd
from functools import reduce
import operator
import pyjsonviewer
import geopandas as gpd
import time

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
def geocode(direccion):
    """
    Esta función saca las coordenadas de la dirección que le pases
    """
    data = requests.get(f"https://geocode.xyz/{direccion}?json=1").json()
    try:
        return {"type": "Point", "coordinates": [data["latt"], data["longt"]]}
    except:
        return data

In [3]:
def getFromDict(diccionario,mapa):
    return reduce(operator.getitem,mapa,diccionario)

In [4]:
def type_point(lista):
    return {"type":"Point", "coordinates": lista}

In [5]:
categories = pd.read_csv("../data/categories.csv")

In [9]:
def categories_dic(cat_list):    
    dicc = {}
    for c in cat_list:
        dicc[c] = categories.categoryId[categories["category"] == c].values[0]
    return dicc

In [10]:
venues_list = ["Design Studio","Nursery School","Preschool","Elementary School","Tech Startup","Coffee Shop",
               "Airport Terminal","Train Station",
              "Nightlife Spot","Vegetarian / Vegan Restaurant","Basketball Stadium",
               "Veterinarian","Pet Service","Pet Store"]  

In [141]:
cat_dic = categories_dic(venues_list)
cat_dic

{'Design Studio': '4bf58dd8d48988d1f4941735',
 'Nursery School': '4f4533814b9074f6e4fb0107',
 'Preschool': '52e81612bcbc57f1066b7a45',
 'Elementary School': '4f4533804b9074f6e4fb0105',
 'Tech Startup': '4bf58dd8d48988d125941735',
 'Coffee Shop': '4bf58dd8d48988d1e0931735',
 'Airport Terminal': '4bf58dd8d48988d1eb931735',
 'Train Station': '4bf58dd8d48988d129951735',
 'Nightlife Spot': '4d4b7105d754a06376d81259',
 'Vegetarian / Vegan Restaurant': '4bf58dd8d48988d1d3941735',
 'Basketball Stadium': '4bf58dd8d48988d18b941735',
 'Veterinarian': '4d954af4a243a5684765b473',
 'Pet Service': '5032897c91d4c4b30a586d69',
 'Pet Store': '4bf58dd8d48988d100951735'}

In [138]:
def localizations_coor(dicc):
    local = {}
    for city,loc in localizations_dicc.items():
        local[city] = geocode(loc)["coordinates"]
        time.sleep(15)
    return local
    

In [139]:
localizations_dicc={"Austin":"Texas Capitol, Austin, Texas","Hamburg":"Hamburg city center,Hamburg,Germany","Newcastle":"Newcastle upon Tyne, United Kingdom"}
localizations = localizations_coor(localizations_dicc)
localizations

{'Austin': ['30.35471', '-97.69344'],
 'Hamburg': ['53.56250', '9.95730'],
 'Newcastle': ['54.96890', '-1.61764']}

In [12]:
#  localizations = {'Austin': ['30.35471', '-97.69344'],
#  'Hamburg': ['53.56250', '9.95730'],
#  'Newcastle': ['54.96890', '-1.61764']}

In [140]:
localizations

{'Austin': ['30.35471', '-97.69344'],
 'Hamburg': ['53.56250', '9.95730'],
 'Newcastle': ['54.96890', '-1.61764']}

In [14]:
# Austin = {"Austin":[30.268735, -97.745209]}
# Hamburg = {"Hamburg" : [53.5973055, 9.9762519]}
# Newcastle = {"Newcastle": [54.971346, -1.611141]}
# localizations = {"Austin":[30.268735, -97.745209],"Hamburg" : [53.5973055, 9.9762519],"Newcastle": [54.971346, -1.611141]}


In [222]:
def place_store(localizations,cat_dic):
    url_query = 'https://api.foursquare.com/v2/venues/search'
    client_id = os.getenv("four")
    client_secret = os.getenv("foursec")
    
    venues = []
    for loc,coor in localizations.items():
        for cat,cat_id in cat_dic.items():
            time.sleep(1)
            parametros={
                "client_id": client_id,
                "client_secret": client_secret,
                "v": "20180323",
                "ll": f"{coor[0]}, {coor[1]}",
                "categoryId":cat_id,
                "radius":2000
            }
            
            venues.append(requests.get(url_query, params = parametros).json()["response"]["venues"])
           
            if cat == "Pet Store":
                print(f"{loc} done")
    print("all done")
           

    return venues

In [208]:
#total_venues = place_store(localizations,cat_dic)

In [163]:
venues = [ven for venues in total_venues for ven in venues]

In [164]:
# todo = {"nombre": ["name"],"city":["location","city"],"country":["location","country"],"category":["categories",0,"name"],
#        "latitud": ["location", "lat"], "longitud": ["location", "lng"],"distance":["location","distance"]}

In [165]:
def extraetodo(json):
    total = []
    try:
        todo = {"nombre": ["name"], #"city":["location","city"],
                "country":["location","country"],
                "category":["categories",0,"name"],
                "latitud": ["location", "lat"], "longitud": ["location", "lng"],"distance":["location","distance"]}
            
        for elemento in json:
            store = {key: getFromDict(elemento,value) for key,value in todo.items()}
            store["location"] = type_point([store["latitud"], store["longitud"]])
            total.append(store)

        print("done")
    except:
        
        print(f"{json.index(elemento)} wrong")
    
    return total

In [166]:
distances = extraetodo(venues)

done


In [167]:
#venues[4] #no city, cñono incluirla??

In [188]:
distances[55]

{'nombre': 'Studioseminar',
 'country': 'Deutschland',
 'category': 'Design Studio',
 'latitud': 53.568866,
 'longitud': 9.98105,
 'distance': 1722,
 'location': {'type': 'Point', 'coordinates': [53.568866, 9.98105]}}

In [169]:
dist_df = pd.DataFrame(distances)

In [171]:
#borrar todos los cafés que no sean starbucks
no_starbucks = list(dist_df.index[(dist_df["category"] == "Coffee Shop") & (dist_df["nombre"] != "Starbucks")])
dist_df.drop(no_starbucks,axis=0, inplace = True)

In [172]:
dist_df[(dist_df["category"] == "Coffee Shop") & (dist_df["nombre"] != "Starbucks")]

,nombre,country,category,latitud,longitud,distance,location


In [173]:
dist_df.country.unique()

array(['United States', 'Deutschland', 'United Kingdom'], dtype=object)

In [219]:
dist_df.category.unique()

array(['Design Studio', 'Preschool', 'Elementary School', 'Tech Startup',
       'Bar', 'American Restaurant', 'Wings Joint', 'Sports Bar',
       'Wine Bar', 'Korean Restaurant', 'Restaurant',
       'Mexican Restaurant', 'Nightclub', 'Strip Club',
       'Vegetarian / Vegan Restaurant', 'Veterinarian', 'Pet Service',
       'Nursery School', 'Daycare', 'College Classroom',
       'Miscellaneous Shop', 'Office', 'Deli / Bodega', 'Coffee Roaster',
       'Platform', 'Train Station', 'Beer Bar', 'Hookah Bar', 'Brewery',
       'Beach Bar', 'Café', 'German Restaurant', 'Pub',
       'Tapas Restaurant', 'Cocktail Bar', 'Gastropub', 'Hotel Bar',
       'Burrito Place', 'Donut Shop', 'Indian Restaurant',
       'Cigkofte Place', 'Organic Grocery', 'Falafel Restaurant',
       'Pet Store', 'Art Gallery', 'Coffee Shop', 'Bookstore',
       'Indie Movie Theater', 'Gay Bar', 'Burger Joint',
       'Fast Food Restaurant', 'Food & Drink Shop', 'Athletics & Sports'],
      dtype=object)

In [226]:
dist_df[dist_df["category"] == "American Restaurant"]

,nombre,country,category,latitud,longitud,distance,location
26,Applebee's Grill + Bar,United States,American Restaurant,30.337639,-97.702053,2072,"{'type': 'Point', 'coordinates': [30.33763898,..."


In [174]:
Austin_df = dist_df[["nombre","category","distance","location"]][dist_df["country"] == "United States"].sort_values(["category","distance"])
Austin_df.head(3)

,nombre,category,distance,location
26,Applebee's Grill + Bar,American Restaurant,2072,"{'type': 'Point', 'coordinates': [30.33763898,..."
25,Chabelo's Bar,Bar,276,"{'type': 'Point', 'coordinates': [30.352285385..."
28,Jekyll And Hyde Billards,Bar,1023,"{'type': 'Point', 'coordinates': [30.363741920..."


In [175]:
Hamburg_df = dist_df[["nombre","category","distance","location"]][dist_df["country"] == "Deutschland"].sort_values(["category","distance"])
Hamburg_df.head(3)

,nombre,category,distance,location
213,Froggy's,Bar,497,"{'type': 'Point', 'coordinates': [53.566964, 9..."
211,Rosi’s Bar,Bar,1377,"{'type': 'Point', 'coordinates': [53.550362812..."
214,Down Under,Bar,1781,"{'type': 'Point', 'coordinates': [53.564131584..."


In [176]:
Newcastle_df = dist_df[["nombre","category","distance","location"]][dist_df["country"] == "United Kingdom"].sort_values(["category","distance"])
Newcastle_df.head(3)

,nombre,category,distance,location
276,Northern Print,Art Gallery,1699,"{'type': 'Point', 'coordinates': [54.975296818..."
416,Northumbria University Sport Central,Athletics & Sports,1262,"{'type': 'Point', 'coordinates': [54.978317593..."
378,The Victoria Comet,Bar,35,"{'type': 'Point', 'coordinates': [54.969207500..."


In [ ]:
Newcastle_df.category